<a href="https://colab.research.google.com/github/lawrencejesse/2023-2034-Lawrence-Ranch-NDVI/blob/main/Sentinel2_RasterExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Packaging and Download

### Subtask:
Zip the exported GeoTIFFs in the `/content/exports` directory and provide a download link for the zip file.

**Reasoning**:
Use the `zip` command to create a zip archive of the exported files and then provide a link to download the created zip file.

In [1]:
import os
from google.colab import files

def zip_and_download_exports():
    export_dir = '/content/exports'
    zip_filename = 'ndvi_exports.zip'
    zip_filepath = f'/content/{zip_filename}'

    if os.path.exists(export_dir) and os.listdir(export_dir):
        # Create a zip archive of the exported files
        !zip -r $zip_filepath $export_dir

        # Provide a download link for the zip file
        print(f"Your files are ready for download. Click the link below to download {zip_filename}")
        files.download(zip_filepath)
    else:
        print("No files found in the export directory to zip.")

# You can call this function after the export tasks are complete and the files are in the /content/exports directory.
# Note: Earth Engine exports to Drive are asynchronous. You'll need to wait for them to complete before zipping.
# A more robust solution would involve monitoring the tasks, but for simplicity, you can manually run this cell after exports are finished.

## Data Export

### Subtask:
Implement the export logic, including handling individual image export and optional mean NDVI export.

**Reasoning**:
Define a function `export_ndvi` that takes the processed collection, AOI, start date, end date, and the mean NDVI toggle status as input. Inside the function, check if the mean NDVI toggle is on. If so, compute the mean NDVI for the collection and export it. Otherwise, iterate through the collection and export each image individually, clipping to the AOI and using descriptive filenames.

In [3]:
import os
import ee.batch

def export_ndvi(processed_collection, aoi, start_date, end_date, export_mean=False):
    """
    Exports the processed NDVI images as GeoTIFFs.

    Args:
        processed_collection (ee.ImageCollection): The processed Sentinel-2 image collection with NDVI.
        aoi (ee.FeatureCollection or ee.Geometry): The area of interest.
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.
        export_mean (bool): Whether to export the mean NDVI for the date range.
    """
    export_dir = '/content/exports'
    if not os.path.exists(export_dir):
        os.makedirs(export_dir)

    if export_mean:
        print("Exporting mean NDVI...")
        mean_ndvi_image = processed_collection.mean().clip(aoi)
        filename = f'NDVI_Mean_{start_date.replace("-", "")}_{end_date.replace("-", "")}.tif'
        task = ee.batch.Export.image.toDrive(
            image=mean_ndvi_image,
            description=filename,
            folder='earth_engine_exports', # You can change this folder name in your Google Drive
            fileNamePrefix=filename.replace('.tif', ''),
            scale=10,
            region=aoi.geometry(),
            fileFormat='GeoTIFF',
            crs='EPSG:4326'
        )
        task.start()
        print(f"Mean NDVI export task started: {task.id}")
    else:
        print("Exporting individual NDVI images...")
        image_list = processed_collection.toList(processed_collection.size())

        for i in range(image_list.size().getInfo()):
            image = ee.Image(image_list.get(i)).clip(aoi)
            image_date = ee.Image(image_list.get(i)).date().format('YYYYMMDD').getInfo()
            filename = f'NDVI_{image_date}.tif' # AOINAME is not available, using date only
            task = ee.batch.Export.image.toDrive(
                image=image,
                description=filename,
                folder='earth_engine_exports', # You can change this folder name in your Google Drive
                fileNamePrefix=filename.replace('.tif', ''),
                scale=10,
                region=aoi.geometry(),
                fileFormat='GeoTIFF',
                crs='EPSG:4326'
            )
            task.start()
            print(f"Export task for {filename} started: {task.id}")

## Sentinel-2 Data Processing

### Subtask:
Define a function to build the Sentinel-2 collection, apply a cloud mask using the SCL band, and compute the NDVI.

**Reasoning**:
Define a function `process_sentinel2` that takes the AOI, start date, and end date as input. Inside the function, filter the Sentinel-2 collection by date and bounds, apply the cloud mask, and compute the NDVI.

In [4]:
def process_sentinel2(aoi, start_date, end_date):
    """
    Builds the Sentinel-2 collection, applies cloud masking, and computes NDVI.

    Args:
        aoi (ee.FeatureCollection or ee.Geometry): The area of interest.
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.

    Returns:
        ee.ImageCollection: The processed Sentinel-2 image collection with NDVI.
    """
    # Filter Sentinel-2 SR Harmonized collection by date and bounds
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(aoi)

    # Function to mask clouds using the Sentinel-2 SCL band
    def mask_clouds(image):
        scl = image.select('SCL')
        # These are the pixel values for clouds, cloud shadows, and snow.
        cloud_mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
        return image.updateMask(cloud_mask)

    # Apply cloud mask and compute NDVI
    def compute_ndvi(image):
        # Scale reflectance bands
        scaled_image = image.select(['B4', 'B8']).multiply(0.0001)
        ndvi = scaled_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        return image.addBands(ndvi)

    processed_collection = collection.map(mask_clouds).map(compute_ndvi)

    return processed_collection.select('NDVI')

# Task
Build a Google Colab notebook in Python that allows users to upload a polygon file (KML, GeoJSON, or zipped Shapefile), parse it as an AOI in WGS84, preview the AOI on a map, select a date range using widgets, download cloud-masked Sentinel-2 NDVI GeoTIFFs for the AOI and date range (clipped to AOI if possible), save them with descriptive filenames to `/content/exports`, zip the files, and provide a download link. Include an optional toggle to export a single mean NDVI for the date range. The notebook should use `earthengine-api`, `geemap`, and `ipywidgets`, authenticate with Earth Engine, use Sentinel-2 SR Harmonized (COPERNICUS/S2_SR_HARMONIZED) with SCL cloud masking, compute NDVI after scaling reflectance by 0.0001, and output GeoTIFFs at 10m scale in EPSG:4326.

## Setup and authentication

### Subtask:
Install necessary libraries (`earthengine-api`, `geemap`, `ipywidgets`), authenticate with Earth Engine, and initialize it.


**Reasoning**:
Install the necessary libraries using pip.



In [5]:
%pip install -U -q earthengine-api geemap ipywidgets

**Reasoning**:
Authenticate and initialize Earth Engine.



In [6]:
import ee

try:
    ee.Initialize()
    print('Google Earth Engine has been successfully initialized.')
except Exception as e:
    print(f'Failed to initialize Google Earth Engine: {e}')
    print('Please authenticate with Google Earth Engine by running ee.Authenticate() in a separate cell.')


Failed to initialize Google Earth Engine: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.
Please authenticate with Google Earth Engine by running ee.Authenticate() in a separate cell.


**Reasoning**:
Authenticate Earth Engine as instructed by the error message.



In [7]:
ee.Authenticate()

True

**Reasoning**:
Re-initialize Earth Engine after successful authentication.



In [8]:
try:
    ee.Initialize()
    print('Google Earth Engine has been successfully initialized after authentication.')
except Exception as e:
    print(f'Failed to initialize Google Earth Engine after authentication: {e}')

Failed to initialize Google Earth Engine after authentication: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.


**Reasoning**:
Re-initialize Earth Engine with a project ID as instructed by the error message.



In [9]:
try:
    ee.Initialize(project='jessemapping')
    print('Google Earth Engine has been successfully initialized with a project.')
except Exception as e:
    print(f'Failed to initialize Google Earth Engine with a project: {e}')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


Google Earth Engine has been successfully initialized with a project.


## Aoi upload and visualization

### Subtask:
Create functionality to upload a polygon file (KML, GeoJSON, or zipped Shapefile), parse it as an Earth Engine feature collection in WGS84, and display it on an interactive map using `geemap` with a satellite basemap, ensuring it's centered and visible.


**Reasoning**:
Import necessary libraries and create a file upload widget.



In [24]:
import ipywidgets as widgets
import geemap
import os

# Define the handle_upload function here
def handle_upload(change):
    print("handle_upload function triggered.") # Debug print
    uploaded_file = upload_widget.value[0]
    file_name = uploaded_file['name']
    file_content = uploaded_file['content']

    global aoi # Declare aoi as global
    aoi = None # Reset aoi
    print(f"Processing file: {file_name}") # Debug print

    try:
        if file_name.endswith('.kml'):
            # Save content to a temporary KML file
            with open(file_name, 'wb') as f:
                f.write(file_content)
            aoi = geemap.kml_to_ee(file_name)
            os.remove(file_name) # Clean up temporary file
        elif file_name.endswith('.geojson'):
            # Save content to a temporary GeoJSON file
            with open(file_name, 'wb') as f:
                f.write(file_content)
            aoi = geemap.geojson_to_ee(file_name)
            os.remove(file_name) # Clean up temporary file
        elif file_name.endswith('.zip'):
            # Save content to a temporary zip file
            with open(file_name, 'wb') as f:
                f.write(file_content)
            aoi = geemap.shp_to_ee(file_name)
            os.remove(file_name) # Clean up temporary file
        else:
            print("Unsupported file type.")
            return

        if aoi:
            # Ensure the AOI is in WGS84 (EPSG:4326) with a non-zero error margin
            aoi = aoi.geometry().transform('EPSG:4326', 1) # Added maxError=1
            print("AOI uploaded and processed successfully.") # Debug print
            print("Run the next cell to display the map.")
        else:
            print("Failed to process the uploaded file: AOI is None.") # Debug print

    except Exception as e:
        print(f"An error occurred during file processing: {e}") # Debug print
        aoi = None # Ensure aoi is None if processing fails


upload_widget = widgets.FileUpload(
    accept='.kml,.geojson,.zip',
    multiple=False
)

display(upload_widget)

# Attach the observer to the widget
upload_widget.observe(handle_upload, names='value')

FileUpload(value=(), accept='.kml,.geojson,.zip', description='Upload')

handle_upload function triggered.
Processing file: LAWRENCE_RANCH_PLUS.kml
AOI uploaded and processed successfully.
Run the next cell to display the map.


**Reasoning**:
Define the function to handle file uploads, parse the file as an Earth Engine feature collection, and display it on a map.



In [22]:
# This cell is no longer needed as handle_upload and the observer are in the same cell as the widget.

**Reasoning**:
Add a new cell to display the map with the uploaded AOI after the file has been processed by the `handle_upload` function.

In [26]:
# Display the AOI on a map after upload
if 'aoi' in globals() and aoi is not None:
    m = geemap.Map(basemap='SATELLITE')
    m.addLayer(ee.FeatureCollection(aoi), {}, 'Uploaded AOI')
    m.centerObject(aoi)
    display(m)
else:
    print("Please upload an AOI using the widget above first.")

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

## Interactive widgets for date and export

### Subtask:
Add `ipywidgets` for selecting a start and end date and a button to trigger the data export process. Include a toggle for exporting the mean NDVI.


**Reasoning**:
Create and display the date picker widgets, the mean NDVI checkbox, and the export button as specified in the instructions.



In [27]:
from datetime import datetime
import ipywidgets as widgets

start_date_widget = widgets.DatePicker(
    description='Start Date:',
    disabled=False,
    value=datetime(2023, 1, 1)
)

end_date_widget = widgets.DatePicker(
    description='End Date:',
    disabled=False,
    value=datetime(2023, 12, 31)
)

mean_ndvi_toggle = widgets.Checkbox(
    value=False,
    description='Export Mean NDVI Only',
    disabled=False,
    indent=False
)

export_button = widgets.Button(
    description='Start Export',
    disabled=False,
    button_style='',
    tooltip='Click to start the export process',
    icon='download'
)

display(start_date_widget, end_date_widget, mean_ndvi_toggle, export_button)

# Attach the export function to the button click event
def on_export_button_clicked(b):
    if 'aoi' not in globals():
        print("Please upload an AOI first.")
        return

    start_date_str = start_date_widget.value.strftime('%Y-%m-%d')
    end_date_str = end_date_widget.value.strftime('%Y-%m-%d')
    export_mean = mean_ndvi_toggle.value

    processed_collection = process_sentinel2(aoi, start_date_str, end_date_str)

    export_ndvi(processed_collection, aoi, start_date_str, end_date_str, export_mean)

export_button.on_click(on_export_button_clicked)

DatePicker(value=datetime.datetime(2023, 1, 1, 0, 0), description='Start Date:', step=1)

DatePicker(value=datetime.datetime(2023, 12, 31, 0, 0), description='End Date:', step=1)

Checkbox(value=False, description='Export Mean NDVI Only', indent=False)

Button(description='Start Export', icon='download', style=ButtonStyle(), tooltip='Click to start the export pr…

Exporting individual NDVI images...


AttributeError: 'Geometry' object has no attribute 'geometry'

# Task
Build a Google Colab notebook in Python that allows users to upload a polygon file (KML, GeoJSON, or zipped Shapefile), parse it as an AOI in WGS84 (EPSG:4326), preview the AOI on a simple map with a Google Satellite basemap, select a date range using widgets, filter for cloud-free Sentinel-2 images within the AOI and date range, display the dates of available images for user selection, download the selected cloud-masked Sentinel-2 NDVI GeoTIFFs clipped to the AOI (or filtered by bounds), save the GeoTIFFs with descriptive filenames to `/content/exports`, zip the exported files, and provide a download link. The notebook should use `earthengine-api`, `geemap`, and `ipywidgets`, authenticate with Earth Engine, use Sentinel-2 SR Harmonized (COPERNICUS/S2_SR_HARMONIZED) and mask clouds with SCL, compute NDVI ((B8-B4)/(B8+B4)) after scaling reflectance by 0.0001, use a scale of 10m and EPSG:4326 output. Optionally, include a toggle to export a single mean NDVI for the selected dates.

## Interactive widgets for date selection

### Subtask:
Add `ipywidgets` for selecting a start and end date.


**Reasoning**:
The traceback indicates that the `aoi` object, which is an `ee.Geometry`, does not have a `.geometry()` method called in the `export_ndvi` function. The `ee.batch.Export.image.toDrive` function expects a `region` parameter which should be an `ee.Geometry`. Since the `aoi` is already an `ee.Geometry` after being transformed in the `handle_upload` function, calling `.geometry()` on it is incorrect. I will correct the call in the `export_ndvi` function.



In [33]:
from datetime import datetime
import ipywidgets as widgets

start_date_widget = widgets.DatePicker(
    description='Start Date:',
    disabled=False,
    value=datetime(2023, 1, 1)
)

end_date_widget = widgets.DatePicker(
    description='End Date:',
    disabled=False,
    value=datetime(2023, 12, 31)
)

mean_ndvi_toggle = widgets.Checkbox(
    value=False,
    description='Export Mean NDVI Only',
    disabled=False,
    indent=False
)

export_button = widgets.Button(
    description='Start Export',
    disabled=False,
    button_style='',
    tooltip='Click to start the export process',
    icon='download'
)

display(start_date_widget, end_date_widget, mean_ndvi_toggle, export_button)

# Attach the export function to the button click event
def on_export_button_clicked(b):
    if 'aoi' not in globals() or aoi is None:
        print("Please upload an AOI first.")
        return

    start_date_str = start_date_widget.value.strftime('%Y-%m-%d')
    end_date_str = end_date_widget.value.strftime('%Y-%m-%d')
    export_mean = mean_ndvi_toggle.value

    processed_collection = process_sentinel2(aoi, start_date_str, end_date_str)

    export_ndvi(processed_collection, aoi, start_date_str, end_date_str, export_mean)

export_button.on_click(on_export_button_clicked)

# Correct the export_ndvi function to use aoi directly as region
def export_ndvi(processed_collection, aoi, start_date, end_date, export_mean=False):
    """
    Exports the processed NDVI images as GeoTIFFs.

    Args:
        processed_collection (ee.ImageCollection): The processed Sentinel-2 image collection with NDVI.
        aoi (ee.FeatureCollection or ee.Geometry): The area of interest.
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.
        export_mean (bool): Whether to export the mean NDVI for the date range.
    """
    export_dir = '/content/exports'
    if not os.path.exists(export_dir):
        os.makedirs(export_dir)

    if export_mean:
        print("Exporting mean NDVI...")
        mean_ndvi_image = processed_collection.mean().clip(aoi)
        filename = f'NDVI_Mean_{start_date.replace("-", "")}_{end_date.replace("-", "")}.tif'
        task = ee.batch.Export.image.toDrive(
            image=mean_ndvi_image,
            description=filename,
            folder='earth_engine_exports', # You can change this folder name in your Google Drive
            fileNamePrefix=filename.replace('.tif', ''),
            scale=10,
            region=aoi, # Corrected: Use aoi directly as it's already a Geometry
            fileFormat='GeoTIFF',
            crs='EPSG:4326'
        )
        task.start()
        print(f"Mean NDVI export task started: {task.id}")
    else:
        print("Exporting individual NDVI images...")
        image_list = processed_collection.toList(processed_collection.size())

        for i in range(image_list.size().getInfo()):
            image = ee.Image(image_list.get(i)).clip(aoi)
            image_date = ee.Image(image_list.get(i)).date().format('YYYYMMDD').getInfo()
            filename = f'NDVI_{image_date}.tif' # AOINAME is not available, using date only
            task = ee.batch.Export.image.toDrive(
                image=image,
                description=filename,
                folder='earth_engine_exports', # You can change this folder name in your Google Drive
                fileNamePrefix=filename.replace('.tif', ''),
                scale=10,
                region=aoi, # Corrected: Use aoi directly as it's already a Geometry
                fileFormat='GeoTIFF',
                crs='EPSG:4326'
            )
            task.start()
            print(f"Export task for {filename} started: {task.id}")

DatePicker(value=datetime.datetime(2023, 1, 1, 0, 0), description='Start Date:', step=1)

DatePicker(value=datetime.datetime(2023, 12, 31, 0, 0), description='End Date:', step=1)

Checkbox(value=False, description='Export Mean NDVI Only', indent=False)

Button(description='Start Export', icon='download', style=ButtonStyle(), tooltip='Click to start the export pr…

Exporting individual NDVI images...
Export task for NDVI_202505125.tif started: DBJ5NNHYGSXQ7OPYPEKE2WLJ
Export task for NDVI_202505130.tif started: KRCBVDQRTLCN4CHX6OD4WK6H
Export task for NDVI_202505132.tif started: W46QAAJ5TYY7GN5SBBZUNKJE
Export task for NDVI_202505135.tif started: 3LT74ZSXFK27ZPBESB7XKTHX
Export task for NDVI_202505140.tif started: H4NY54AKJWBERTHK5N56ONIZ
Export task for NDVI_202505145.tif started: XMYXY3AJFK26JO5EUPEHZ6LA
Export task for NDVI_202505150.tif started: FUDQ54JQP3IWG7IDTZTLGL4Q
Export task for NDVI_202506152.tif started: DXIS5FQBXPVHEDOUVENO5BI5
Export task for NDVI_202506155.tif started: NLCJTSPHIB2PHA4KU3FCYXIY
Export task for NDVI_202506160.tif started: MFKWDMP64DDZKJ7ZDRO6GHG3
Export task for NDVI_202506165.tif started: UORP3D656U27NZU33FWKDQRE
Export task for NDVI_202506170.tif started: 334O7MG3KY6TS2BMGTJEUNNT
Export task for NDVI_202506172.tif started: N442QQW6VUBNUO6VMSCC4WCU
Export task for NDVI_202506175.tif started: TFLBME55POTGBCSGPHQX2UQ

## Filter and display available dates

### Subtask:
Filter the Sentinel-2 collection by the selected date range and AOI, apply the cloud mask, and extract the dates of the available cloud-free images. Display these dates to the user, possibly using a selectable list or checkboxes.


**Reasoning**:
Define the `get_available_dates` function to filter the Sentinel-2 collection by date and AOI, apply the cloud mask, and extract the dates of the available images. Then, modify the `on_export_button_clicked` function to call this function and display the dates using an Output widget.



## Select dates for export

### Subtask:
Allow the user to select which of the available dates they want to export.


**Reasoning**:
Modify the `on_export_button_clicked` function to display a `SelectMultiple` widget with the available dates, and create a new button `process_selected_button` that will handle the export based on the selected dates.



## Packaging and download

### Subtask:
Zip the exported GeoTIFFs in the `/content/exports` directory and provide a download link for the zip file.


**Reasoning**:
Call the zip_and_download_exports() function to zip the exported files and initiate the download.



## Packaging and download

### Subtask:
Retry zipping the exported GeoTIFFs in the `/content/exports` directory and provide a download link for the zip file.


**Reasoning**:
Call the `zip_and_download_exports()` function again to attempt to zip the exported files and provide a download link. This step is a direct retry of the previous failed attempt, assuming the user might have manually transferred files to the `/content/exports` directory or the Earth Engine exports have completed and been transferred by another mechanism (which is outside the scope of this automated process but necessary for the function to succeed).



## Summary:

### Data Analysis Key Findings

*   Interactive date pickers and a toggle for mean NDVI export were successfully implemented using `ipywidgets`.
*   The capability to filter Sentinel-2 images by AOI and date range, apply a cloud mask, and retrieve the dates of available cloud-free images was added.
*   An interactive `SelectMultiple` widget was introduced to allow users to choose specific dates for export from the list of available dates.
*   The export logic was modified to process and export either individual NDVI images or the mean NDVI based on the dates selected by the user.
*   Attempts to zip and provide a download link for exported files from the `/content/exports` directory failed because no files were found in the directory, indicating that the asynchronous Earth Engine export tasks to Google Drive likely did not complete or were not transferred back to the Colab environment before the zipping step.

### Insights or Next Steps

*   The asynchronous nature of Earth Engine exports to Google Drive requires implementing a mechanism to monitor task completion and transfer the files to the Colab environment's local storage (`/content/exports`) *before* attempting to zip them.
*   Consider adding visual feedback or a progress bar to indicate the status of Earth Engine export tasks to the user.
